# Season 4 Barbarian Bash/Berserk Ripping gear optimization

### This notebook calculates the optimal assignment of stats to gear affixes, temper affixes and optimal gems for the Bash Bleed build. I am making some important assumptions:
- Paingorger does not interact with Adaptability and Edgemaster (source: beatdropper on Mobalytics)
- Berserk Ripping does not interact with Adaptability, Moonrise, Edgemaster (source: icytroll on Discord)
- For the Gushing Wounds multiplier, 2.1*(1+csd)-0.4 where csd is the raw Critical Strike damage, which is further multiplied only by Heavy Handed and Grandfather if applicable.
- Attack Speed is treated as a global multiplier, disregarding breakpoints. To the best of my knowledge, nobody has shared frame data for Bash, so I cannot implement attack speed breakpoints. Take results w.r.t. attack speed affixes with a grain of salt. Also attack speed has more value than just dps (faster Hectic procs, vuln uptime etc.), that are not modeled here.
- Vulnerable uptime is assumed to be perfect and not modeled here. Might change soon
- Assuming Paingorger works with most multipliers that are not Adaptability and Edgemaster (e.g. Counteroffensive, Pit Fighter, Moonrise). If that is not the case then Paingorger's are a bit overvalued in my model
- Optimizing with Grandfather does not account for the loss of a legendary aspect, which should be kept in mind for comparison to the non-Grandfather setting.

Consequently, calculations are without using the Adaptability Aspect, Edgemaster Aspect, and the Moonrise Aspect is moved to a one-handed weapon.

TODO: masterworking crits, greater affixes?

### To get results that fit your character and preferences, adjust the values in the Parameters section. For example, enter your Paingorger stats, certain Paragon values, talent point allocations etc., and then execute all cells in order. Gear affixes on two-handed weapons count as two affixes in terms of the solution.

In [25]:
import pyomo.environ as pyo

In [26]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

## Parameters

In [27]:
# all additive damage that is not bleeding damage, damage while berserking, vulnerable damage, critical strike damage
dmg_add = 1.356 + 1.05 + 0.2 + 0.09
dmg_bleed = 0.709
dmg_bers_p = 1.4                    # damage while berserking from paragon
dmg_vuln_p = 0.959                   # vulnerable damage from paragon
dmg_csd_ip = 0.175+0.175+0.35+1.158  # csd from weapon implicits and paragon
paingorger_roll = 2
paingorger_as = 0.145
paingorger_csc = 0.116
# enter your strength from all sources but rings and amulet (including strength on weapons), assume 2H slashing is not Grandfather
str_no_jewelery = 2220
# all attack speed bonuses from non-gear bonuses, here: Rapid, Moonrise, Carnage, Rupture
as_non_gear = 0.3 + 0.2 + 0.16 + 0.4
# critical strike chance from base and stats, not counting No Mercy which does not apply to Berserk Ripping bleeds
csc_non_gear = 0.05 + 0.1

# passive skill ranks in talents for Heavy Handed (hh), Cut to the Bone (cttb), Counteroffensive (co), Pit Fighter (pf)
ranks_hh = 3
ranks_cttb = 3
ranks_co = 3
ranks_pf = 3

# maximal number of skill rank affixes on the amulet. Set to 1 if you are not RMT'ing
amu_max_skillrank_affixes = 1
# set to 1 if you want to use cooldown reduction on the amulet, otherwise 0
amu_use_cdr = 1
# set to 1 if you want to use the lucky hit: chance to make enemy vulnerable affix on a ring, otherwise 0
use_lh_vuln = 1
# set to 1 if you use Berserk Ripping, 0 otherwise
use_berserk_ripping = 1
# set to 1 if you want to use Grandfather, 0 otherwise
use_gf = 1

## Constants

In [28]:
# data for all the affix values, assuming max roll and fully masterworked (1.45), but no mw crits

temper_csd = 0.85 * 1.45
temper_vd = 0.55 * 1.45
temper_bd = 0.55 * 1.45
temper_bash_cleave = 1.05 * 1.45

gear_csd = 0.5 * 1.45
gear_vd = 0.4 * 1.45
gear_csc = 0.06 * 1.45
gear_as = 0.09 * 1.45
gear_str = 90 * 1.45

amulet_csc = 0.08 * 1.45
amulet_as = 0.09 * 1.45
amulet_str_perc = 0.073 * 1.45
amulet_skill_ranks = 3           # 2 skill ranks upgrade to 3 without mw crit but full mw

gem_csd = 0.25
gem_vd = 0.2
gem_basicd = 0.45

gf_alldmg = 0.56 * 1.45

## Model starts here

In [29]:
model = pyo.ConcreteModel()

In [30]:
# variables indication how many gear/temper affixes you should take for each stat, affixes on two-handed weapons count as 2
# vd=vulnerable damage, bd=berserking damage, csd=critical strike damage
#model.x_vd = pyo.Var(domain=pyo.NonNegativeReals)
#model.x_csd = pyo.Var(domain=pyo.NonNegativeReals)
#model.x_bd = pyo.Var(domain=pyo.NonNegativeReals)

model.x_gear_vd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gear_csd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gear_csc = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gear_as = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gear_str = pyo.Var(domain=pyo.NonNegativeIntegers)

model.x_temper_csd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_temper_vd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_temper_bd = pyo.Var(domain=pyo.NonNegativeIntegers)

# extra variables for amulet gear affixes
model.x_amu_csc = pyo.Var(domain=pyo.Binary)
model.x_amu_str = pyo.Var(domain=pyo.Binary)
model.x_amu_as = pyo.Var(domain=pyo.Binary)
model.x_amu_hh = pyo.Var(domain=pyo.Binary)
model.x_amu_cttb = pyo.Var(domain=pyo.Binary)
model.x_amu_co = pyo.Var(domain=pyo.Binary)
model.x_amu_pf = pyo.Var(domain=pyo.Binary)

amu_var_list = [model.x_amu_csc, model.x_amu_str, model.x_amu_as, model.x_amu_hh, model.x_amu_cttb, model.x_amu_co, model.x_amu_pf]

# variables for gems number of each respective gem
model.x_gem_csd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gem_vd = pyo.Var(domain=pyo.NonNegativeIntegers)
model.x_gem_basicd = pyo.Var(domain=pyo.NonNegativeIntegers)

# introduce an auxiliary variable for berserking damage that caps at 3 for the purpose of calculating Blood Rage multiplier
model.x_bd_capped = pyo.Var(domain=pyo.NonNegativeReals)

#### In the cell below, you can manually add stats for your greater affixes or for testing in general. For example add 0.1 to csc to simulate having 10% more Critical Strike Chance, and after the optimization compare the objective function value to the previous one without the added 10% Critical Strike Chance.

In [31]:
# combine parameters with variables so the final damage formula is less verbose
pg = 1 + paingorger_roll
hh = 1 + 0.05*(ranks_hh+model.x_amu_hh*amulet_skill_ranks)
cttb = 1 + 0.05*(ranks_cttb+model.x_amu_cttb*amulet_skill_ranks)
co = 1 + 0.04*(ranks_co+model.x_amu_co*amulet_skill_ranks)
pf = 1 + 0.03*(ranks_pf+model.x_amu_pf*amulet_skill_ranks)

vd = dmg_vuln_p + model.x_gear_vd*gear_vd + model.x_temper_vd*temper_vd + model.x_gem_vd*gem_vd
csd = dmg_csd_ip + model.x_gear_csd*gear_csd + model.x_temper_csd*temper_csd + model.x_gem_csd*gem_csd
bd = dmg_bers_p + model.x_temper_bd*temper_bd
bd_capped = model.x_bd_capped
# add includes all additive damage multipliers that are not vd, csd, bd and bleeding damage. add_bld includes bleeding damage
add = dmg_add + model.x_gem_basicd*gem_basicd + use_gf*gf_alldmg
add_bld = add + dmg_bleed + use_gf*gf_alldmg   # all damage% doubledips on bleeds

as_multi = 1 + paingorger_as + as_non_gear + model.x_gear_as*gear_as + model.x_amu_as*amulet_as
csc = csc_non_gear + paingorger_csc + model.x_gear_csc*gear_csc + model.x_amu_csc*amulet_csc + 0.025*use_gf + 0.3

str_multi = 1 + (str_no_jewelery + model.x_gear_str*gear_str - 247*use_gf)*0.1*0.01*(1+model.x_amu_str*amulet_str_perc)

In [32]:
# subexpressions for the objective function

# subexpression for the hit damage part of bash, assuming Adaptability and Moonrise(1.8) do not interact with Berserk Ripping
# assuming vulnerable damage works with Paingorger, but crit does not
# 1.15 Two-Handed Mace Expertise
bash_hit = 1.8 * (1+bd_capped*0.1) * ((1-csc+pg)*(1+add+vd+bd) + csc*1.5*hh*1.15*(1+add+vd+csd+bd)*(1+use_gf))

# subexpression for the bleed caused by Berserk Ripping
# multipliers that apply to both normal bleeds and gushing wounds bleeds already included in the base
br_base = (1+pg) * 0.6 * (1+vd*0.15) * (1+bd_capped*0.1) * (1+add_bld+vd+bd) * cttb
gushing_wounds = 1 + ((1+csd)*1.5*hh*(1+use_gf) - 1) * 1.4
bash_bleed = (1-csc)*br_base + csc*br_base*gushing_wounds

In [33]:
# objective function maximizes the total multiplier incl. attack speed, assuming enemies are always vulnerable (1.2 multiplier irrelevant for the optimization)
total_bash_multi = bash_hit + bash_bleed*use_berserk_ripping
model.obj = pyo.Objective(expr = as_multi * str_multi * (1+(6-2*use_gf)*temper_bash_cleave) * total_bash_multi * co * pf, sense=pyo.maximize)

## Constraints
# constraint for maximum number of temperings total on gear
model.temper = pyo.Constraint(expr = model.x_temper_vd+model.x_temper_csd+model.x_temper_bd <= 9)

# constraints for maximum number of gear affixes
# model.gear = pyo.Constraint(expr = model.x_gear_vd+model.x_gear_csd <= 8)
model.gear1 = pyo.Constraint(expr = model.x_gear_vd+model.x_gear_csd <= 8 - (2*use_gf))
model.gear2 = pyo.Constraint(expr = model.x_gear_csc <= 2)
model.gear3 = pyo.Constraint(expr = model.x_gear_as <= 2)
model.gear4 = pyo.Constraint(expr = model.x_gear_str <= 2)
model.gear5 = pyo.Constraint(expr = model.x_gear_vd+model.x_gear_csd+model.x_gear_as+model.x_gear_str+model.x_gear_csc <= 12-use_lh_vuln-(2*use_gf))

# amulet constraints, max number of gear affixes and max number of skill rank affixes
model.amu1 = pyo.Constraint(expr = sum(amu_var_list) <= 3-amu_use_cdr)
model.amu2 = pyo.Constraint(expr = model.x_amu_hh+model.x_amu_cttb+model.x_amu_co+model.x_amu_pf <= amu_max_skillrank_affixes)

# limit the capped berserking damage variable to the uncapped one, and limit it to 3
model.bd1 = pyo.Constraint(expr = bd_capped <= bd)
model.bd2 = pyo.Constraint(expr = bd_capped <= 3)

# constraint for maximum number of gems
model.gems = pyo.Constraint(expr = model.x_gem_csd+model.x_gem_vd+model.x_gem_basicd <= 6)

In [34]:
model.pprint()

19 Var Declarations
    x_amu_as : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_co : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_csc : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_cttb : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_hh : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_pf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    x_amu_str : Size=1, Index

In [35]:
if 'google.colab' in sys.modules:
    opt = pyo.SolverFactory('couenne')
else:
    opt = pyo.SolverFactory('couenne', executable = 'C:/Users/Daniel/idaes_opti_venv/solvers/couenne.exe')
    
results = opt.solve(model, tee=True)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -342359.25       36 0.008
NLP0014I             2      INFEAS 0.99999994       16 0.004
Loaded instance "C:\Users\Daniel\AppData\Local\Temp\tmpgui7l0xz.pyomo.nl"
Constraints:           11
Variables:             19 (18 integer)
Auxiliaries:           41 (1 integer)

Coin0506I Presolve 92 (-12) rows, 47 (-13) columns and 304 (-19) elements
Clp0006I 0  Obj -16358.805 Primal inf 5.6746 (2) Dual inf 398.34471 (1)
Clp0006I 37  Obj -3243391.3 Primal inf 1035.8178 (30)
Clp0006I 68  Obj -1373469.8 Primal inf 2.8520345 (8)
Clp0006I 75  Obj -1341891.1
Clp0000I Optimal - objective value -1341891.1
Clp0032I Optimal objective -1341891.053 - 75 iterations time 0.002, Presolve 0.00
Cl

In [36]:
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('Found optimal solution, with the following variable values:')
else:
    print('No optimal solution found, but these are the best variable assignments the solver found:')

total_vd = dmg_vuln_p + model.x_gear_vd.value*gear_vd + model.x_temper_vd.value*temper_vd + model.x_gem_vd.value*gem_vd
total_csd = dmg_csd_ip + model.x_gear_csd.value*gear_csd + model.x_temper_csd.value*temper_csd + model.x_gem_csd.value*gem_csd
total_bd = dmg_bers_p + model.x_temper_bd.value*temper_bd

total_hh = 1 + 0.05*(ranks_hh+model.x_amu_hh.value*amulet_skill_ranks)

print('Total Vulnerable Damage:', total_vd)
print('Total (raw) Critical Strike Damage:', total_csd)
print('Total Berserking Damage:', total_bd, '\n')
print('Total Critical Strike Chance: ', csc_non_gear + paingorger_csc + model.x_gear_csc.value*gear_csc + model.x_amu_csc.value*amulet_csc +0.025*use_gf)
print('Total Attack Speed', paingorger_as + as_non_gear + model.x_gear_as.value*gear_as + model.x_amu_as.value*amulet_as, '\n')

print('Ratio of CSD to VD (raw values):', total_csd/total_vd)
print('Ratio of CSD to VD (stat sheet):', ((1+total_csd)*1.5*total_hh*(1+use_gf)-1) / ((total_vd * 1.2) + 0.2))
print('Objective function value: ', pyo.value(model.obj))
print('\n')

print('Number of vuln temper affixes:', model.x_temper_vd.value)
print('Number of csd temper affixes:', model.x_temper_csd.value)
print('Number of berserking damage temper affixes:', model.x_temper_bd.value)
print('Number of vuln gear affixes:', model.x_gear_vd.value)
print('Number of csd gear affixes:', model.x_gear_csd.value)
print('Number of csc gear affixes:', model.x_gear_csc.value)
print('Number of as gear affixes:', model.x_gear_as.value)
print('Number of str gear affixes:', model.x_gear_str.value, '(excluding weapons)')
print('Using \"Lucky Hit: Chance to Vuln\" affix on ring: ', bool(use_lh_vuln))
print('\n')

amu_str_list = ['Critical Strike Chance', 'STR%', 'Attack Speed', 'Heavy Handed', 'Cut to the Bone', 'Counteroffensive', 'Pit Fighter']
sol_amu_affixes = [amu_str_list[i] for i in range(len(amu_var_list)) if amu_var_list[i].value>=0.6]
if amu_use_cdr:
    sol_amu_affixes.append('CDR')
print('Amulet gear affixes: ', sol_amu_affixes, '\n')

print('Number of VD gems:', model.x_gem_vd.value)
print('Number of CSD gems:', model.x_gem_csd.value)
print('Number of Basic Skill Damage gems:', model.x_gem_basicd.value)

Found optimal solution, with the following variable values:
Total Vulnerable Damage: 5.836499999999999
Total (raw) Critical Strike Damage: 9.253
Total Berserking Damage: 2.995 

Total Critical Strike Chance:  0.581
Total Attack Speed 1.3355000000000001 

Ratio of CSD to VD (raw values): 1.5853679431165941
Ratio of CSD to VD (stat sheet): 5.411963130569977
Objective function value:  341415.3391622317


Number of vuln temper affixes: 1.0
Number of csd temper affixes: 6.0
Number of berserking damage temper affixes: 2.0
Number of vuln gear affixes: 5.999999999999999
Number of csd gear affixes: 0.0
Number of csc gear affixes: 2.0
Number of as gear affixes: 0.9999999999999997
Number of str gear affixes: 0.0 (excluding weapons)
Using "Lucky Hit: Chance to Vuln" affix on ring:  True


Amulet gear affixes:  ['Critical Strike Chance', 'Heavy Handed', 'CDR'] 

Number of VD gems: 3.0
Number of CSD gems: 0.0
Number of Basic Skill Damage gems: 2.9999999999999933


In [314]:
print(pyo.value(model.obj))

230791.28619198478
